In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-energy-behavior-of-prosumers/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv
/kaggle/input/predict-energy-behavior-of-prosumers/public_timeseries_testing_util.py
/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/county_id_to_name_map.json
/kaggle/input/predict-energy-behavior-of-prosumers/train.csv
/kaggle/input/predict-energy-behavior-of-prosumers/forecast_weather.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/sample_submission.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/client.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/gas_prices.csv
/kaggle/input/predict-energy-behavior-of-prosumers/example_test_files/electricity

In [2]:
train = "/kaggle/input/predict-energy-behavior-of-prosumers/train.csv"
gas = '/kaggle/input/predict-energy-behavior-of-prosumers/gas_prices.csv'
elect = '/kaggle/input/predict-energy-behavior-of-prosumers/electricity_prices.csv'
client = '/kaggle/input/predict-energy-behavior-of-prosumers/client.csv'

In [3]:
train_df = pd.read_csv(train)
gas_df = pd.read_csv(gas)
elect_df = pd.read_csv(elect)
client_df = pd.read_csv(client)

In [4]:
train_df

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2
...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,2023-05-31 23:00:00,637,2018347,64
2018348,15,1,1,0.000,0,2023-05-31 23:00:00,637,2018348,59
2018349,15,1,1,28.404,1,2023-05-31 23:00:00,637,2018349,59
2018350,15,1,3,0.000,0,2023-05-31 23:00:00,637,2018350,60


In [ ]:
del train_df client_df

In [5]:
train_df.isna().sum()

county                  0
is_business             0
product_type            0
target                528
is_consumption          0
datetime                0
data_block_id           0
row_id                  0
prediction_unit_id      0
dtype: int64

In [6]:
train_df['target'].fillna(train_df['target'].mean(), inplace=True)

In [7]:
def split_datetime(df, date_time_column):
    # Convert the date_time column to datetime
    df[date_time_column] = pd.to_datetime(df[date_time_column])

    # Split the date and time into separate columns
    df['date'] = df[date_time_column].dt.date
    df['time'] = df[date_time_column].dt.time

    # Delete the old column
    df.drop(date_time_column, axis=1, inplace=True)

    return df

In [8]:
gas_df.head()

,forecast_date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id
0,2021-09-01,45.23,46.32,2021-08-31,1
1,2021-09-02,45.62,46.29,2021-09-01,2
2,2021-09-03,45.85,46.40,2021-09-02,3
3,2021-09-04,46.30,46.80,2021-09-03,4
4,2021-09-05,46.30,46.58,2021-09-04,5


In [9]:
split_datetime(train_df,'datetime')

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,date,time
0,0,0,1,0.713,0,0,0,0,2021-09-01,00:00:00
1,0,0,1,96.590,1,0,1,0,2021-09-01,00:00:00
2,0,0,2,0.000,0,0,2,1,2021-09-01,00:00:00
3,0,0,2,17.314,1,0,3,1,2021-09-01,00:00:00
4,0,0,3,2.904,0,0,4,2,2021-09-01,00:00:00
...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,637,2018347,64,2023-05-31,23:00:00
2018348,15,1,1,0.000,0,637,2018348,59,2023-05-31,23:00:00
2018349,15,1,1,28.404,1,637,2018349,59,2023-05-31,23:00:00
2018350,15,1,3,0.000,0,637,2018350,60,2023-05-31,23:00:00


In [10]:
train_df = pd.merge(train_df,gas_df[["lowest_price_per_mwh","highest_price_per_mwh","data_block_id"]],on='data_block_id',how='inner')

In [11]:
split_datetime(elect_df,'forecast_date')

,euros_per_mwh,origin_date,data_block_id,date,time
0,92.51,2021-08-31 00:00:00,1,2021-09-01,00:00:00
1,88.90,2021-08-31 01:00:00,1,2021-09-01,01:00:00
2,87.35,2021-08-31 02:00:00,1,2021-09-01,02:00:00
3,86.88,2021-08-31 03:00:00,1,2021-09-01,03:00:00
4,88.43,2021-08-31 04:00:00,1,2021-09-01,04:00:00
...,...,...,...,...,...
15281,82.10,2023-05-29 19:00:00,637,2023-05-30,19:00:00
15282,150.85,2023-05-29 20:00:00,637,2023-05-30,20:00:00
15283,82.10,2023-05-29 21:00:00,637,2023-05-30,21:00:00
15284,82.09,2023-05-29 22:00:00,637,2023-05-30,22:00:00


In [12]:
train_df = pd.merge(train_df,elect_df[["euros_per_mwh","data_block_id","time"]],on=['data_block_id',"time"],how='inner')

In [13]:
train_df

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,date,time,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh
0,0,0,1,1.687,0,1,2928,0,2021-09-02,00:00:00,45.23,46.32,92.51
1,0,0,1,109.366,1,1,2929,0,2021-09-02,00:00:00,45.23,46.32,92.51
2,0,0,2,0.000,0,1,2930,1,2021-09-02,00:00:00,45.23,46.32,92.51
3,0,0,2,21.008,1,1,2931,1,2021-09-02,00:00:00,45.23,46.32,92.51
4,0,0,3,1.003,0,1,2932,2,2021-09-02,00:00:00,45.23,46.32,92.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015151,15,1,0,197.233,1,637,2018347,64,2023-05-31,23:00:00,29.00,34.00,-1.29
2015152,15,1,1,0.000,0,637,2018348,59,2023-05-31,23:00:00,29.00,34.00,-1.29
2015153,15,1,1,28.404,1,637,2018349,59,2023-05-31,23:00:00,29.00,34.00,-1.29
2015154,15,1,3,0.000,0,637,2018350,60,2023-05-31,23:00:00,29.00,34.00,-1.29


In [14]:
client_df

,product_type,county,eic_count,installed_capacity,is_business,date,data_block_id
0,1,0,108,952.89,0,2021-09-01,2
1,2,0,17,166.40,0,2021-09-01,2
2,3,0,688,7207.88,0,2021-09-01,2
3,0,0,5,400.00,1,2021-09-01,2
4,1,0,43,1411.00,1,2021-09-01,2
...,...,...,...,...,...,...,...
41914,1,15,51,415.60,0,2023-05-29,637
41915,3,15,161,2035.75,0,2023-05-29,637
41916,0,15,15,620.00,1,2023-05-29,637
41917,1,15,20,624.50,1,2023-05-29,637


In [15]:
merged_df = pd.merge(train_df,client_df[["product_type","eic_count","installed_capacity","is_business","data_block_id","county"]],on=['data_block_id',"product_type","county","is_business"],how='inner')

In [16]:
merged_df

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,date,time,lowest_price_per_mwh,highest_price_per_mwh,euros_per_mwh,eic_count,installed_capacity
0,0,0,1,0.793,0,2,5856,0,2021-09-03,00:00:00,45.62,46.29,96.99,108,952.89
1,0,0,1,107.129,1,2,5857,0,2021-09-03,00:00:00,45.62,46.29,96.99,108,952.89
2,0,0,1,2.106,0,2,5978,0,2021-09-03,01:00:00,45.62,46.29,94.77,108,952.89
3,0,0,1,81.920,1,2,5979,0,2021-09-03,01:00:00,45.62,46.29,94.77,108,952.89
4,0,0,1,0.955,0,2,6100,0,2021-09-03,02:00:00,45.62,46.29,93.88,108,952.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009441,15,1,3,265.328,1,637,2018091,60,2023-05-31,21:00:00,29.00,34.00,82.10,55,2188.20
2009442,15,1,3,0.001,0,637,2018220,60,2023-05-31,22:00:00,29.00,34.00,82.09,55,2188.20
2009443,15,1,3,274.569,1,637,2018221,60,2023-05-31,22:00:00,29.00,34.00,82.09,55,2188.20
2009444,15,1,3,0.000,0,637,2018350,60,2023-05-31,23:00:00,29.00,34.00,-1.29,55,2188.20


In [17]:
weather="/kaggle/input/predict-energy-behavior-of-prosumers/historical_weather.csv"

In [18]:
hw_df = pd.read_csv(weather)

### mapping latitude and longitude to county 

In [19]:
hw_df

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
0,2021-09-01 00:00:00,14.4,12.0,0.0,0.0,1015.8,4,4,0,0,6.694444,3,0.0,0.0,0.0,57.6,21.7,1
1,2021-09-01 00:00:00,14.0,12.0,0.0,0.0,1010.6,7,8,0,0,4.944444,353,0.0,0.0,0.0,57.6,22.2,1
2,2021-09-01 00:00:00,14.4,12.8,0.0,0.0,1014.9,6,7,0,0,5.833333,348,0.0,0.0,0.0,57.6,22.7,1
3,2021-09-01 00:00:00,15.4,13.0,0.0,0.0,1014.4,4,2,4,0,7.111111,349,0.0,0.0,0.0,57.6,23.2,1
4,2021-09-01 00:00:00,15.9,12.6,0.0,0.0,1013.8,12,7,0,20,8.388889,360,0.0,0.0,0.0,57.6,23.7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710795,2023-05-30 10:00:00,10.8,4.1,0.4,0.0,1018.7,40,0,67,0,1.333333,283,440.0,264.0,176.0,59.7,26.2,637
1710796,2023-05-30 10:00:00,11.4,2.9,0.3,0.0,1018.8,41,0,69,0,1.527778,302,453.0,283.0,170.0,59.7,26.7,637
1710797,2023-05-30 10:00:00,9.5,3.2,0.1,0.0,1018.9,47,0,79,0,3.416667,302,477.0,312.0,165.0,59.7,27.2,637
1710798,2023-05-30 10:00:00,11.0,1.6,0.0,0.0,1018.6,31,0,52,0,3.472222,309,535.0,400.0,135.0,59.7,27.7,637


In [20]:
unique_pairs = hw_df[["latitude", 'longitude']].drop_duplicates()

In [21]:
unique_pairs

,latitude,longitude
0,57.6,21.7
1,57.6,22.2
2,57.6,22.7
3,57.6,23.2
4,57.6,23.7
...,...,...
107,59.7,26.2
108,59.7,26.7
109,59.7,27.2
110,59.7,27.7


In [ ]:
unique_pairs['County_Numbers'].nunique()

In [ ]:
 hw_df['latitude'].nunique()

In [22]:
import pandas as pd
from geopy.geocoders import Nominatim

# Initialize Nominatim
geolocator = Nominatim(user_agent="my_app")

# Iterate through each row in the DataFrame
for index, row in unique_pairs.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']

    # Perform reverse geocoding to get county information
    location = geolocator.reverse((latitude, longitude))

    if location is not None:
        county = location.raw['address'].get('county', None)

        # Add county information to the DataFramemaakond
        if county is not None:
            unique_pairs.loc[index, 'county_name'] = county

# Print the updated DataFrame
unique_pairs


,latitude,longitude,county_name
0,57.6,21.7,Ventspils novads
1,57.6,22.2,Ventspils novads
2,57.6,22.7,NaN
3,57.6,23.2,Saare maakond
4,57.6,23.7,NaN
...,...,...,...
107,59.7,26.2,Lääne-Viru maakond
108,59.7,26.7,Lääne-Viru maakond
109,59.7,27.2,Кингисеппский район
110,59.7,27.7,Кингисеппский район


In [24]:
unique_pairs['county_code']=unique_pairs['county_name'].map({v: k for k, v in county_numbers.items()})

In [ ]:
unique_pairs = unique_pairs.drop(columns='county')

In [28]:
unique_pairs = unique_pairs.dropna()

In [29]:
unique_pairs 

,latitude,longitude,county_name,county_code
3,57.6,23.2,Saare maakond,10
10,57.6,26.7,Võru maakond,15
11,57.6,27.2,Võru maakond,15
14,57.9,21.7,Saare maakond,10
15,57.9,22.2,Saare maakond,10
...,...,...,...,...
91,59.4,25.2,Harju maakond,0
92,59.4,25.7,Harju maakond,0
104,59.7,24.7,Harju maakond,0
105,59.7,25.2,Harju maakond,0


In [30]:
df_merged = pd.merge(hw_df,unique_pairs, on=['latitude', 'longitude'], how='inner')

unique_pairs

In [35]:
df_merged

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,county
0,2021-09-01 00:00:00,15.4,13.0,0.0,0.0,1014.4,4,2,4,0,7.111111,349,0.0,0.0,0.0,57.6,23.2,1,10
1,2021-09-01 01:00:00,15.6,13.1,0.0,0.0,1014.0,1,1,0,0,7.444444,349,0.0,0.0,0.0,57.6,23.2,1,10
2,2021-09-01 02:00:00,14.9,12.6,0.0,0.0,1014.4,25,8,0,58,8.888889,352,0.0,0.0,0.0,57.6,23.2,1,10
3,2021-09-01 03:00:00,14.9,12.3,0.0,0.0,1014.3,17,8,0,32,9.055556,354,0.0,0.0,0.0,57.6,23.2,1,10
4,2021-09-01 04:00:00,14.9,11.9,0.0,0.0,1014.2,37,28,0,38,9.277778,352,0.0,0.0,0.0,57.6,23.2,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992870,2023-05-30 06:00:00,9.1,3.2,0.0,0.0,1018.2,33,0,55,0,2.527778,236,24.0,1.0,23.0,59.7,25.7,637,0
992871,2023-05-30 07:00:00,9.5,2.9,0.0,0.0,1018.2,32,7,42,0,2.277778,241,101.0,23.0,78.0,59.7,25.7,637,0
992872,2023-05-30 08:00:00,10.3,3.1,0.6,0.0,1018.4,49,12,64,0,2.444444,251,228.0,117.0,111.0,59.7,25.7,637,0
992873,2023-05-30 09:00:00,10.7,4.6,0.1,0.0,1018.5,44,3,69,0,2.250000,249,303.0,163.0,140.0,59.7,25.7,637,0


In [36]:
split_datetime(df_merged,"datetime")

,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,county,date,time
0,15.4,13.0,0.0,0.0,1014.4,4,2,4,0,7.111111,349,0.0,0.0,0.0,57.6,23.2,1,10,2021-09-01,00:00:00
1,15.6,13.1,0.0,0.0,1014.0,1,1,0,0,7.444444,349,0.0,0.0,0.0,57.6,23.2,1,10,2021-09-01,01:00:00
2,14.9,12.6,0.0,0.0,1014.4,25,8,0,58,8.888889,352,0.0,0.0,0.0,57.6,23.2,1,10,2021-09-01,02:00:00
3,14.9,12.3,0.0,0.0,1014.3,17,8,0,32,9.055556,354,0.0,0.0,0.0,57.6,23.2,1,10,2021-09-01,03:00:00
4,14.9,11.9,0.0,0.0,1014.2,37,28,0,38,9.277778,352,0.0,0.0,0.0,57.6,23.2,1,10,2021-09-01,04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992870,9.1,3.2,0.0,0.0,1018.2,33,0,55,0,2.527778,236,24.0,1.0,23.0,59.7,25.7,637,0,2023-05-30,06:00:00
992871,9.5,2.9,0.0,0.0,1018.2,32,7,42,0,2.277778,241,101.0,23.0,78.0,59.7,25.7,637,0,2023-05-30,07:00:00
992872,10.3,3.1,0.6,0.0,1018.4,49,12,64,0,2.444444,251,228.0,117.0,111.0,59.7,25.7,637,0,2023-05-30,08:00:00
992873,10.7,4.6,0.1,0.0,1018.5,44,3,69,0,2.250000,249,303.0,163.0,140.0,59.7,25.7,637,0,2023-05-30,09:00:00


In [31]:
df_merged = df_merged.rename(columns={'county_code':'county'})

In [32]:
df_merged = df_merged.drop(columns='county_name')

In [33]:
from fastai.tabular.all  import add_datepart

In [50]:
merged_df = add_datepart(merged_df,'date')

/opt/conda/lib/python3.10/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


In [53]:
merged_df['time'] = merged_df['time'].astype(str)
merged_df[['hours', 'minutes', 'seconds']] = merged_df['time'].str.split(':', expand=True)

In [47]:
df_merged = df_merged['time'].drop(columns='time')

In [45]:
df_merged['hours']

0         00
1         01
2         02
3         03
4         04
          ..
992870    06
992871    07
992872    08
992873    09
992874    10
Name: hours, Length: 992875, dtype: object

In [ ]:
def process_time(df, time):
    df[time] = pd.to_datetime(df[time])

    # Split the date and time into separate columns
    df['hour'] = df[time].dt.date
    df['min'] = df[time].dt.time
    df['sec'] = df[time]

    # Delete the old column
    df.drop(date_time_column, axis=1, inplace=True)

    return df

In [38]:
df_merged['county']= df_merged['county'].astype("int64")

In [39]:
merged_df= pd.merge(merged_df,df_merged, on=['date', 'time','county'], how='inner')

In [ ]:
merged_df = merged_df.drop(columns='county_name')

In [52]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9359034 entries, 0 to 9359033
Data columns (total 44 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   county                  int64  
 1   is_business             int64  
 2   product_type            int64  
 3   target                  float64
 4   is_consumption          int64  
 5   data_block_id_x         int64  
 6   row_id                  int64  
 7   prediction_unit_id      int64  
 8   time                    object 
 9   lowest_price_per_mwh    float64
 10  highest_price_per_mwh   float64
 11  euros_per_mwh           float64
 12  eic_count               int64  
 13  installed_capacity      float64
 14  temperature             float64
 15  dewpoint                float64
 16  rain                    float64
 17  snowfall                float64
 18  surface_pressure        float64
 19  cloudcover_total        int64  
 20  cloudcover_low          int64  
 21  cloudcover_mid          int64  

In [56]:
from sklearn.preprocessing import LabelEncoder

In [57]:
le = LabelEncoder()

In [ ]:
merged_df = merged_df['time'].drop(columns='time')

In [58]:
       
    
merged_df[['Is_month_end', 'Is_month_start','Is_quarter_end','Is_quarter_start','Is_year_end','Is_year_start']] = merged_df[['Is_month_end', 'Is_month_start','Is_quarter_end','Is_quarter_start','Is_year_end','Is_year_start']].apply(lambda x: le.fit_transform(x))

In [65]:
del train_df, client_df,elect_df ,gas_df ,df_merged

NameError: name 'train_df' is not defined

In [67]:
merged_df = merged_df.drop(columns='time')

In [ ]:
df_merged = df_merged.dropna()

In [ ]:
df_merged['country_code']

In [ ]:
df_merged  

df_merged[['hours', 'minutes', 'seconds']] = df_merged['time'].str.split(':', expand=True)

In [61]:
from sklearn import preprocessing

In [68]:
min_max_scaler = preprocessing.MinMaxScaler()
df_normalized = pd.DataFrame(min_max_scaler.fit_transform(merged_df), columns=merged_df.columns)


,\

In [69]:
df_normalized.head()

,county,is_business,product_type,target,is_consumption,data_block_id_x,row_id,prediction_unit_id,lowest_price_per_mwh,highest_price_per_mwh,...,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,hours,minutes,seconds
0,0.0,0.0,0.333333,0.000051,0.0,0.0,0.0,0.0,0.078954,0.044998,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.333333,0.000051,0.0,0.0,0.0,0.0,0.078954,0.044998,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.333333,0.000051,0.0,0.0,0.0,0.0,0.078954,0.044998,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.333333,0.000051,0.0,0.0,0.0,0.0,0.078954,0.044998,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.333333,0.000051,0.0,0.0,0.0,0.0,0.078954,0.044998,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
del merged_df

NameError: name 'merged_df' is not defined

In [77]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [74]:
X_train, X_test, y_train, y_test = train_test_split(df_normalized.drop('target', axis=1), df_normalized['target'], test_size=0.2, random_state=42)

In [78]:
model = XGBRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.5,
    colsample_bytree=0.5,
    random_state=42
)

In [79]:
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [80]:
# Evaluate the model on the testing data
score = model.score(X_test, y_test)
print(f'Test accuracy: {score:.3f}')

Test accuracy: 0.904


In [ ]:
df['Number'] = df['County'].map({v: k for k, v in county_numbers.items()})

In [ ]:
unique_pairs['country_code'].nunique()

In [23]:
county_numbers = { 
    "0":'Harju maakond',
    "1":'Hiiu maakond',
    "3":"Järva maakond",
    "4":"Jõgeva maakond",
    "6":"Lääne maakond",
    "7":"Pärnu maakond",
    "8":"Põlva maakond",
    "9":"Rapla maakond",
    "10":"Saare maakond",
    "11":"Tartu maakond",
    "12":"nan",
    "13":"Valga maakond",
    "14":"Viljandi maakond",
    "15":"Võru maakond"
}

In [ ]:
county_numbers = {
  "0": "HARJUMAA",   harju maakonda 
  "1": "HIIUMAA",   'Hiiu maakond'
  "2": "IDA-VIRUMAA", 
  "3": "JÄRVAMAA", jarvamaakonda 
  "4": "JÕGEVAMAA",    jogea maakonda 
  "5": "LÄÄNE-VIRUMAA",  
  "6": "LÄÄNEMAA",   Lääne maakond'
  "7": "PÄRNUMAA",   pranu maakonda
  "8": "PÕLVAMAA",   poliva maakonda
  "9": "RAPLAMAA",  raplamaakonda
  "10": "SAAREMAA",  saaremaakonda
  "11": "TARTUMAA",   taru maakonda
  "12": "UNKNOWN",    nan
  "13": "VALGAMAA",    valga maakonda
  "14": "VILJANDIMAA",  viljandi maakonda
  "15": "VÕRUMAA"     'Võru maakond'
}